#  Teste Engenheiro de Dados - Mesha Tecnologia

## Instalação das Dependencias

In [1]:
!pip install pyspark
!pip install findspark
!pip install pandas 
!pip install install-jdk

  Using cached pyspark-3.4.1-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
  Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
  Using cached pandas-2.0.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.2 MB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached numpy-1.25.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.6 MB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
  Using cached install_jdk-1.0.4-py3-none-any.whl (15 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


## Importação das Dependencias e Inicialização do PySpark

In [2]:
import pandas as pd
import findspark
from pyspark.sql import SparkSession

In [3]:
findspark.init()

In [4]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

Picked up JAVA_TOOL_OPTIONS:  -Xmx12884m
Picked up JAVA_TOOL_OPTIONS:  -Xmx12884m
23/06/28 12:15:11 WARN Utils: Your hostname, danilojpfre-testeengenh-349ickl44bp resolves to a loopback address: 127.0.0.1; using 10.0.5.2 instead (on interface eth0)
23/06/28 12:15:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/28 12:15:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
spark

## Extração dos Dados

In [6]:
path = "/workspace/teste-engenheiro-de-dados/dados/row/DADOS/MICRODADOS_ENEM_2020.csv"

In [7]:
enem = spark.read.csv(path, sep=";", header=True)

In [8]:
enem.limit(5).show()

23/06/28 12:15:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+-------------------+---------+---------+----------------------+------------------+---------------+------------------+-------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|T

In [9]:
enem.printSchema()

root
 |-- NU_INSCRICAO: string (nullable = true)
 |-- NU_ANO: string (nullable = true)
 |-- TP_FAIXA_ETARIA: string (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: string (nullable = true)
 |-- TP_COR_RACA: string (nullable = true)
 |-- TP_NACIONALIDADE: string (nullable = true)
 |-- TP_ST_CONCLUSAO: string (nullable = true)
 |-- TP_ANO_CONCLUIU: string (nullable = true)
 |-- TP_ESCOLA: string (nullable = true)
 |-- TP_ENSINO: string (nullable = true)
 |-- IN_TREINEIRO: string (nullable = true)
 |-- CO_MUNICIPIO_ESC: string (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: string (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: string (nullable = true)
 |-- TP_LOCALIZACAO_ESC: string (nullable = true)
 |-- TP_SIT_FUNC_ESC: string (nullable = true)
 |-- CO_MUNICIPIO_PROVA: string (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: string (nullable = tru

In [10]:
enem.where('NU_ANO!="2020"').count()

0

In [11]:
enem.where(enem['NU_NOTA_REDACAO'].isNull()).count()

3028969

## Carregamento na Processing Zone - Seleção de Columns, Correções dos tipos de dados e Transformação para o formato Parquet

### Seleção Columns

In [12]:
enemSelecao = enem.select("NU_INSCRICAO", "TP_SEXO", "TP_COR_RACA", "CO_MUNICIPIO_ESC", "NO_MUNICIPIO_ESC", "SG_UF_ESC", "TP_PRESENCA_CN", "TP_PRESENCA_CH", "TP_PRESENCA_LC", "TP_PRESENCA_MT", "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_REDACAO", "TP_ST_CONCLUSAO")

In [13]:
enemSelecao.limit(5).show()

+------------+-------+-----------+----------------+-------------------+---------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+---------------+---------------+
|NU_INSCRICAO|TP_SEXO|TP_COR_RACA|CO_MUNICIPIO_ESC|   NO_MUNICIPIO_ESC|SG_UF_ESC|TP_PRESENCA_CN|TP_PRESENCA_CH|TP_PRESENCA_LC|TP_PRESENCA_MT|NU_NOTA_CN|NU_NOTA_CH|NU_NOTA_LC|NU_NOTA_MT|NU_NOTA_REDACAO|TP_ST_CONCLUSAO|
+------------+-------+-----------+----------------+-------------------+---------+--------------+--------------+--------------+--------------+----------+----------+----------+----------+---------------+---------------+
|200006271946|      F|          2|            null|               null|     null|             0|             0|             0|             0|      null|      null|      null|      null|           null|              1|
|200001195856|      M|          3|            null|               null|     null|             1|             1|             1|  

### Correção dos tipos dos dados

In [14]:
from pyspark.sql.types import DoubleType, StringType, IntegerType 
from pyspark.sql import functions as f

In [15]:
enemSelecao.printSchema()

root
 |-- NU_INSCRICAO: string (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_COR_RACA: string (nullable = true)
 |-- CO_MUNICIPIO_ESC: string (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_PRESENCA_CN: string (nullable = true)
 |-- TP_PRESENCA_CH: string (nullable = true)
 |-- TP_PRESENCA_LC: string (nullable = true)
 |-- TP_PRESENCA_MT: string (nullable = true)
 |-- NU_NOTA_CN: string (nullable = true)
 |-- NU_NOTA_CH: string (nullable = true)
 |-- NU_NOTA_LC: string (nullable = true)
 |-- NU_NOTA_MT: string (nullable = true)
 |-- NU_NOTA_REDACAO: string (nullable = true)
 |-- TP_ST_CONCLUSAO: string (nullable = true)



In [16]:
enemSelecao.where(enemSelecao['TP_ST_CONCLUSAO'].isNull()).count()

0

In [17]:
enemSelecaoTipos = enemSelecao\
    .withColumn(
    "TP_COR_RACA",
    enemSelecao["TP_COR_RACA"].cast(IntegerType())
    )\
    .withColumn(
    "CO_MUNICIPIO_ESC",
    enemSelecao["CO_MUNICIPIO_ESC"].cast(IntegerType())
    )\
    .withColumn(
    "TP_PRESENCA_CN",
    enemSelecao["TP_PRESENCA_CN"].cast(IntegerType())
    )\
    .withColumn(
    "TP_PRESENCA_CH",
    enemSelecao["TP_PRESENCA_CH"].cast(IntegerType())
    )\
    .withColumn(
    "TP_PRESENCA_LC",
    enemSelecao["TP_PRESENCA_LC"].cast(IntegerType())
    )\
    .withColumn(
    "TP_PRESENCA_MT",
    enemSelecao["TP_PRESENCA_MT"].cast(IntegerType())
    )\
    .withColumn(
    "NU_NOTA_CN",
    enemSelecao["NU_NOTA_CN"].cast(DoubleType())
    )\
    .withColumn(
    "NU_NOTA_CH",
    enemSelecao["NU_NOTA_CH"].cast(DoubleType())
    )\
    .withColumn(
    "NU_NOTA_LC",
    enemSelecao["NU_NOTA_LC"].cast(DoubleType())
    )\
    .withColumn(
    "NU_NOTA_MT",
    enemSelecao["NU_NOTA_MT"].cast(DoubleType())
    )\
    .withColumn(
    "NU_NOTA_REDACAO",
    enemSelecao["NU_NOTA_REDACAO"].cast(IntegerType())
    )\
    .withColumn(
    "TP_ST_CONCLUSAO",
    enemSelecao["TP_ST_CONCLUSAO"].cast(IntegerType())
    )

In [18]:
enemSelecaoTipos.where(enemSelecaoTipos['TP_ST_CONCLUSAO'].isNull()).count()

0

In [19]:
enemSelecaoTipos.printSchema()

root
 |-- NU_INSCRICAO: string (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_PRESENCA_CN: integer (nullable = true)
 |-- TP_PRESENCA_CH: integer (nullable = true)
 |-- TP_PRESENCA_LC: integer (nullable = true)
 |-- TP_PRESENCA_MT: integer (nullable = true)
 |-- NU_NOTA_CN: double (nullable = true)
 |-- NU_NOTA_CH: double (nullable = true)
 |-- NU_NOTA_LC: double (nullable = true)
 |-- NU_NOTA_MT: double (nullable = true)
 |-- NU_NOTA_REDACAO: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)



In [20]:
novosNomes = {'NU_INSCRICAO':'NumInscricao', 'TP_SEXO':'Sexo', 'TP_COR_RACA':'Etnia', 'CO_MUNICIPIO_ESC':'CodMunicipioEsc', 'NO_MUNICIPIO_ESC':'NomMunicipioEsc', 'SG_UF_ESC':'EstadoEsc', 'TP_PRESENCA_CN':'PresencaCN', 'TP_PRESENCA_CH':'PresencaCH', 'TP_PRESENCA_LC':'PresencaLC', 'TP_PRESENCA_MT':'PresencaMT', 'NU_NOTA_CN':'NotaCN', 'NU_NOTA_CH':'NotaCH', 'NU_NOTA_LC':'NotaLC', 'NU_NOTA_MT':'NotaMT', 'NU_NOTA_REDACAO':'NotaRedacao', 'TP_ST_CONCLUSAO':'SituacaoConclusao'}

In [21]:
for key, value in novosNomes.items():
    enemSelecaoTipos = enemSelecaoTipos.withColumnRenamed(key, value)

In [22]:
enemSelecaoTipos.printSchema()

root
 |-- NumInscricao: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Etnia: integer (nullable = true)
 |-- CodMunicipioEsc: integer (nullable = true)
 |-- NomMunicipioEsc: string (nullable = true)
 |-- EstadoEsc: string (nullable = true)
 |-- PresencaCN: integer (nullable = true)
 |-- PresencaCH: integer (nullable = true)
 |-- PresencaLC: integer (nullable = true)
 |-- PresencaMT: integer (nullable = true)
 |-- NotaCN: double (nullable = true)
 |-- NotaCH: double (nullable = true)
 |-- NotaLC: double (nullable = true)
 |-- NotaMT: double (nullable = true)
 |-- NotaRedacao: integer (nullable = true)
 |-- SituacaoConclusao: integer (nullable = true)



### Transformação para o formato Parquet

In [23]:
pathParquet = "/workspace/teste-engenheiro-de-dados/dados/processing"

In [24]:
enemSelecaoTipos.write.parquet(
    pathParquet,
    mode = "overwrite"
)

23/06/28 12:15:54 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/06/28 12:15:54 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/06/28 12:15:54 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/06/28 12:15:54 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/06/28 12:15:54 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/06/28 12:15:54 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 58.46% for 13 writers
23/06/28 12:15:54 WARN MemoryManager: Total allocation exceeds 95.

In [25]:
enemParquet = spark.read.parquet(
    pathParquet
)

In [26]:
enemParquet.limit(5).show()

+------------+----+-----+---------------+---------------+---------+----------+----------+----------+----------+------+------+------+------+-----------+-----------------+
|NumInscricao|Sexo|Etnia|CodMunicipioEsc|NomMunicipioEsc|EstadoEsc|PresencaCN|PresencaCH|PresencaLC|PresencaMT|NotaCN|NotaCH|NotaLC|NotaMT|NotaRedacao|SituacaoConclusao|
+------------+----+-----+---------------+---------------+---------+----------+----------+----------+----------+------+------+------+------+-----------+-----------------+
|200003297356|   F|    2|           null|           null|     null|         0|         0|         0|         0|  null|  null|  null|  null|       null|                1|
|200003373109|   F|    2|           null|           null|     null|         0|         0|         0|         0|  null|  null|  null|  null|       null|                1|
|200004404163|   F|    1|           null|           null|     null|         0|         0|         0|         0|  null|  null|  null|  null|       null

In [27]:
enemParquet.count()

5783109

## Curated Zone - Criando as tabelas para cada pergunta

### Criando uma View

In [28]:
enemParquet.createOrReplaceTempView("enemView")

### 1 - Qual a escola com a maior média de notas?

In [29]:
escolaMaiorMedia = spark.sql("""
        SELECT 
            CodMunicipioEsc, NomMunicipioEsc, 
            EstadoEsc, ROUND(AVG(MediaNotas), 2) AS MediaNotas
        FROM
            (SELECT
                NumInscricao, CodMunicipioEsc, NomMunicipioEsc, EstadoEsc, 
                ROUND((NotaCN+NotaCH+NotaLC+NotaMT+NotaRedacao)/5, 2) AS MediaNotas
            FROM
                enemView
            WHERE 
                    SituacaoConclusao == 2 
                AND 
                    CodMunicipioEsc IS NOT NULL 
                AND 
                    NotaMT IS NOT NULL
                AND
                    NotaCH IS NOT NULL
                AND
                    NotaCN IS NOT NULL
                AND
                    NotaLC IS NOT NULL
                AND 
                    NotaRedacao IS NOT NULL
            )
        GROUP BY CodMunicipioEsc, NomMunicipioEsc, EstadoEsc  
        ORDER BY MediaNotas DESC
        """)

In [30]:
escolaMaiorMedia.limit(10).show()

+---------------+--------------------+---------+----------+
|CodMunicipioEsc|     NomMunicipioEsc|EstadoEsc|MediaNotas|
+---------------+--------------------+---------+----------+
|        3158904| Santana do Manhua�u|       MG|    706.56|
|        4313466|          Novo Xingu|       RS|     704.9|
|        4305850|    Coqueiros do Sul|       RS|    693.52|
|        4117297|      Novo Itacolomi|       PR|     692.7|
|        3107000|           Biquinhas|       MG|    685.64|
|        3147808|         Passa Vinte|       MG|    681.74|
|        3160009|Santo Ant�nio do ...|       MG|     670.8|
|        3170651|Vargem Grande do ...|       MG|    668.77|
|        3149408|      Pedro Teixeira|       MG|    656.16|
|        4304853|        Carlos Gomes|       RS|    654.44|
+---------------+--------------------+---------+----------+



In [31]:
escolaMaiorMedia.count()

5425

### 2 - Qual o aluno com a maior média de notas e o valor dessa média?

In [32]:
alunoMaiorNota = spark.sql("""
        SELECT
            *
        FROM
            (SELECT
                NumInscricao, Sexo, 
                Etnia, NomMunicipioEsc, 
                EstadoEsc, 
                SituacaoConclusao,
                ROUND((NotaCN+NotaCH+NotaLC+NotaMT+NotaRedacao)/5, 2) AS MediaNotas
            FROM
                enemView
            WHERE 
                    NotaMT IS NOT NULL
                AND
                    NotaCH IS NOT NULL
                AND
                    NotaCN IS NOT NULL
                AND
                    NotaLC IS NOT NULL
                AND 
                    NotaRedacao IS NOT NULL)
        ORDER BY MediaNotas DESC
        """)

In [33]:
alunoMaiorNota.limit(10).show()

+------------+----+-----+---------------+---------+-----------------+----------+
|NumInscricao|Sexo|Etnia|NomMunicipioEsc|EstadoEsc|SituacaoConclusao|MediaNotas|
+------------+----+-----+---------------+---------+-----------------+----------+
|200005996961|   M|    1|           null|     null|                1|    858.58|
|200004812135|   M|    1|           null|     null|                1|    852.86|
|200001357436|   M|    1| Rio de Janeiro|       RJ|                2|    852.76|
|200004002694|   M|    1|           null|     null|                1|     851.5|
|200001850224|   F|    1|      S�o Paulo|       SP|                2|    851.34|
|200006051511|   F|    1|    Jo�o Pessoa|       PB|                2|    850.74|
|200003688741|   M|    1|           null|     null|                1|    849.34|
|200006619550|   F|    1|           null|     null|                1|    847.82|
|200001993301|   M|    1|           null|     null|                1|    847.68|
|200004146848|   F|    1|   

#### Dataframe and View Etnia

In [53]:
etniaColumns = ["codEtnia", "nomeEtnia"]
etniaNome = [(0, "Nao declarado"), (1, "Branca"), (2, "Preta"), (3, "Parda"), (4, "Amarela"), (5, "Indigena")]
etniaDataframe = spark.createDataFrame(etniaNome, etniaColumns)
etniaDataframe.show()

+--------+-------------+
|codEtnia|    nomeEtnia|
+--------+-------------+
|       0|Nao declarado|
|       1|       Branca|
|       2|        Preta|
|       3|        Parda|
|       4|      Amarela|
|       5|     Indigena|
+--------+-------------+



In [54]:
etniaDataframe.createOrReplaceTempView("etniaView")

#### Dataframe and View SituacaoConclusao

In [55]:
sitConColumns = ["codCon", "nomeCon"]
sitConNome = [(1, "Concluido"), (2, "Cursando/Conclusao 2020"), (3, "Cursando/Conclusao Apos 2020"), (4, "Sem Conclusao/Previsao")]
sitConDataframe = spark.createDataFrame(sitConNome, sitConColumns)
sitConDataframe.show()

+------+--------------------+
|codCon|             nomeCon|
+------+--------------------+
|     1|           Concluido|
|     2|Cursando/Conclusa...|
|     3|Cursando/Conclusa...|
|     4|Sem Conclusao/Pre...|
+------+--------------------+



In [56]:
sitConDataframe.createOrReplaceTempView("sitConView")

#### Join Tables Etnia e SituaçãoConclusão

In [57]:
alunoMaiorNota = spark.sql("""
        SELECT
            A.NumInscricao, A.Sexo,
            B.nomeEtnia AS Etnia,
            A.NomMunicipioEsc, A.EstadoEsc,
            C.nomeCon AS SituacaoConclusao,
            A.MediaNotas
        FROM
            (SELECT
                NumInscricao, Sexo, 
                Etnia, NomMunicipioEsc, 
                EstadoEsc, 
                SituacaoConclusao,
                ROUND((NotaCN+NotaCH+NotaLC+NotaMT+NotaRedacao)/5, 2) AS MediaNotas
            FROM
                enemView
            WHERE 
                    NotaMT IS NOT NULL
                AND
                    NotaCH IS NOT NULL
                AND
                    NotaCN IS NOT NULL
                AND
                    NotaLC IS NOT NULL
                AND 
                    NotaRedacao IS NOT NULL) A
        INNER JOIN
            etniaView B
            ON codEtnia == Etnia
        INNER JOIN
            sitConView C
            ON codCon == SituacaoConclusao
        ORDER BY MediaNotas DESC
        """)

In [58]:
alunoMaiorNota.limit(10).show()

+------------+----+------+---------------+---------+--------------------+----------+
|NumInscricao|Sexo| Etnia|NomMunicipioEsc|EstadoEsc|   SituacaoConclusao|MediaNotas|
+------------+----+------+---------------+---------+--------------------+----------+
|200005996961|   M|Branca|           null|     null|           Concluido|    858.58|
|200004812135|   M|Branca|           null|     null|           Concluido|    852.86|
|200001357436|   M|Branca| Rio de Janeiro|       RJ|Cursando/Conclusa...|    852.76|
|200004002694|   M|Branca|           null|     null|           Concluido|     851.5|
|200001850224|   F|Branca|      S�o Paulo|       SP|Cursando/Conclusa...|    851.34|
|200006051511|   F|Branca|    Jo�o Pessoa|       PB|Cursando/Conclusa...|    850.74|
|200003688741|   M|Branca|           null|     null|           Concluido|    849.34|
|200006619550|   F|Branca|           null|     null|           Concluido|    847.82|
|200001993301|   M|Branca|           null|     null|           Co

### 3 - Qual a média geral?

In [51]:
mediaGeral = spark.sql("""
        SELECT 
            ROUND(AVG(ROUND((NotaCN+NotaCH+NotaLC+NotaMT+NotaRedacao)/5, 2)), 2) AS MediaGeral
        FROM
            (SELECT 
                *
            FROM
                enemView
            WHERE
                    NotaMT IS NOT NULL
                AND
                    NotaCH IS NOT NULL
                AND
                    NotaCN IS NOT NULL
                AND
                    NotaLC IS NOT NULL
                AND 
                    NotaRedacao IS NOT NULL
            )
        """)

In [52]:
mediaGeral.limit(10).show()

+----------+
|MediaGeral|
+----------+
|    526.58|
+----------+



### 4 - Qual o % de Ausentes?

In [89]:
ausentesEnem = spark.sql("""
            SELECT
                COUNT(NumInscricao) AS NumAusentes
            FROM
                enemView
            WHERE 
            PresencaCN = 0
                OR
            PresencaCH = 0
                OR
            PresencaLC = 0
                OR
            PresencaMT = 0
        """)

In [90]:
ausentesEnem.show()

+-----------+
|NumAusentes|
+-----------+
|    3192751|
+-----------+



In [100]:
totalEnem = spark.sql("""
        SELECT
            COUNT(NumInscricao) AS NumTotal
        FROM
            enemView
        """)

In [101]:
totalEnem.show()

+--------+
|NumTotal|
+--------+
| 5783109|
+--------+



In [102]:
ausentesEnem.createOrReplaceTempView("ausentesEnemView")

In [103]:
totalEnem.createOrReplaceTempView("totalEnemView")

In [109]:
porcentagemAusentesEnem = spark.sql("""
        SELECT
            B.NumTotal, A.NumAusentes, 
            ROUND(((A.NumAusentes*100)/B.NumTotal), 2) AS PorcentagemAusentes
        FROM
            ausentesEnemView AS A,
            totalEnemView AS B
        """)

In [110]:
porcentagemAusentesEnem.show()

+--------+-----------+-------------------+
|NumTotal|NumAusentes|PorcentagemAusentes|
+--------+-----------+-------------------+
| 5783109|    3192751|              55.21|
+--------+-----------+-------------------+



#### Dataframe and View Presenca

In [63]:
presencaColumns = ["codPre", "nomePre"]
presencaNome = [(0, "Faltou"), (1, "Presente"), (2, "Eliminado")]
presencaDataframe = spark.createDataFrame(presencaNome, presencaColumns)
presencaDataframe.show()

+------+---------+
|codPre|  nomePre|
+------+---------+
|     0|   Faltou|
|     1| Presente|
|     2|Eliminado|
+------+---------+



### 5 - Qual o número total de Inscritos?

In [111]:
totalInscritos = spark.sql("""
        SELECT
            NumTotal AS TotalInscritos
        FROM
            totalEnemView 
        """)

In [112]:
totalInscritos.show()

+--------------+
|TotalInscritos|
+--------------+
|       5783109|
+--------------+



### 6 - Qual a média por disciplina?